In [1]:
import numpy as np
import scipy.linalg as LA
from load_mnist import loadImageSet, loadLabelSet

%matplotlib inline
import pylab
import matplotlib.pyplot as plt
from sklearn import neighbors
import pdb


# 数据加载

In [2]:
train_images = loadImageSet('MNIST_data/train-images-idx3-ubyte') 
train_labels = loadLabelSet('MNIST_data/train-labels-idx1-ubyte')

test_images = loadImageSet('MNIST_data/t10k-images-idx3-ubyte') 
test_labels = loadLabelSet('MNIST_data/t10k-labels-idx1-ubyte')

train_x = train_images[0:2000,:] /255.0
train_x = train_x.T
train_y = train_labels[0:2000]

test_x = test_images[0:2000,:] /255.0 
test_x = test_x.T
test_y = test_labels[0:2000]

# 可视化函数

In [3]:
#%%
# 将降维后的数据可视化,2维
def plot_embedding_2d(X, y, title=None, save_prefix=None):
    #坐标缩放到[0,1]区间
    x_min, x_max = np.min(X,axis=0), np.max(X,axis=0)
    X = (X - x_min) / (x_max - x_min)

    #降维后的坐标为（X[i, 0], X[i, 1]），在该位置画出对应的digits
    fig = plt.figure()
#     plt.axis([-2000, 2000,-2000,2000])
    ax = fig.add_subplot(1, 1, 1)
    for i in range(X.shape[0]):
        ax.text(X[i, 0], X[i, 1],str(y[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})

    if title is not None:
        plt.title(title)
        plt.savefig(save_prefix+'_'+title.replace(' ','_')+".png") 

# 进行LE降维需要的函数

In [4]:
def EuclideanDistances(A, B):
    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A,BT)
    # print(vecProd)
    SqA =  A**2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B**2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))    
    SqED = sumSqBEx + sumSqAEx - 2*vecProd
    SqED[SqED<0]=0.0   
    ED = np.array(np.sqrt(SqED))
    return ED               

In [5]:
def get_neighbors_ind_and_distance(X, K=10):
    """
    Param:
        X: M * N matrix
        Y: matrix
        K: The number of neighbors of each point
    """    
    M, N = X.shape
#     X2 = np.sum(X**2, axis=0, keepdims=True)
#     distance = np.tile(X2,(N,1))+np.tile(X2.T,(1,N))-2 * np.matmul(X.T, X)
    distance = EuclideanDistances(X.T, X.T)
    index = np.argsort(distance, axis=0)
    neighborhood = index[1:(1+K),:]
    return neighborhood, distance

In [7]:
def get_weight(neighborhood, distance, t=5.0):
    N = distance.shape[0]
    Weight = np.zeros((N,N))
    for i in xrange(N):
        d = distance[i][neighborhood[:,i]]
        Weight[i][neighborhood[:,i]]= np.exp(-d**2/(4*t))
    return Weight  

In [8]:
def get_degree_and_Laplacian_matrix(W):
    D = np.sum(W, axis=1) ### 按行相加
    D = np.diag(D)
    L = D - W 
    return D, L

In [9]:
def get_embedding(L,D, d):
#     N = M.shape[0]
#     eigenValues, eigenVectors = LA.eig(M)
    eigenvalue, eigenvector_right = LA.eig(L, D)
    eigValInd=np.argsort(eigenvalue)
    eigValInd=eigValInd[1:(d+1)]
    Y_T = eigenvector_right[:, eigValInd]
    Y = Y_T.T
    return Y

# 进行LE降维

In [ ]:
K_list = [5,10,15]
t_list=[5.0, 25.0, 100000.0]
for K in K_list:
    for t in t_list:
        train_x_neighbor, train_distance = get_neighbors_ind_and_distance(train_x, K=K)
        # train_distance = train_distance /train_distance.max()
        train_W = get_weight(train_x_neighbor, train_distance, t=t)
        D, L = get_degree_and_Laplacian_matrix(train_W)
        embedding_train = get_embedding(L, D, d=2)
        tr_Y = embedding_train[:,0:1000]
        te_Y = embedding_train[:,1000:2000]
        ''''' 训练KNN分类器 '''  
        clf = neighbors.KNeighborsClassifier(algorithm='auto',n_neighbors=1, weights= 'distance')  
        clf.fit(tr_Y.T, train_y[0:1000]) 
        """测试准确率"""
        acc = np.float(sum(train_y[1000:2000]==clf.predict(te_Y.T))) / len(train_y[1000:2000])
        print ("N:{}, t:{}, acc:{}".format(K,t, acc))
#         plot_embedding_2d(embedding_train.T, train_y, title="N=%d t=%f" %(K, t), save_prefix='train')

N:5, t:5.0, acc:0.745
N:5, t:25.0, acc:0.717
N:5, t:100000.0, acc:0.712
N:10, t:5.0, acc:0.606
N:10, t:25.0, acc:0.612
N:10, t:100000.0, acc:0.604
N:15, t:5.0, acc:0.601
N:15, t:25.0, acc:0.624
N:15, t:100000.0, acc:0.606


In [10]:
K_list = [5,10,15]
t_list=[5.0, 25.0, 100000.0]
for K in K_list:
    for t in t_list:
        test_x_neighbor, test_distance = get_neighbors_ind_and_distance(test_x, K=K)
        # train_distance = train_distance /train_distance.max()
        test_W = get_weight(test_x_neighbor, test_distance, t=t)
        D, L = get_degree_and_Laplacian_matrix(test_W)
        embedding_test = get_embedding(L, D, d=2)
        tr_Y = embedding_test[:,0:1000]
        te_Y = embedding_test[:,1000:2000]
        ''''' 训练KNN分类器 '''  
        clf = neighbors.KNeighborsClassifier(algorithm='auto',n_neighbors=1, weights= 'distance')  
        clf.fit(tr_Y.T, test_y[0:1000]) 
        """测试准确率"""
        acc = np.float(sum(test_y[1000:2000]==clf.predict(te_Y.T))) / len(test_y[1000:2000])
        print ("N:{}, t:{}, acc:{}".format(K,t, acc))
#         plot_embedding_2d(embedding_test.T, test_y, title="N=%d t=%f" %(K, t), save_prefix='test')

/usr/local/lib/python2.7/dist-packages/numpy/core/numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:373: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


N:5, t:5.0, acc:0.611
N:5, t:25.0, acc:0.612
N:5, t:100000.0, acc:0.628
N:10, t:5.0, acc:0.6
N:10, t:25.0, acc:0.594
N:10, t:100000.0, acc:0.572
N:15, t:5.0, acc:0.568
N:15, t:25.0, acc:0.547
N:15, t:100000.0, acc:0.546
